In [1]:
# Load the .pickle containing the dictionary for the protobuf message

import pickle
with open('metadata.pickle', 'rb') as file:
    session_dict = pickle.load(file)

In [2]:
# Read dictionary from pickle and create protobuf metadata-message

import metadata_pb2
from datetime import datetime
import pytz

# Create SESSION protobuff metadata
sess = metadata_pb2.Session()

if 'bird_type' in session_dict: # UNKNOWN_BIRDTYPE(0), ZEBRA(1), STARLING(2), BENGALESE(3)
    if   session_dict['bird_type'] == 'STARLING':  sess.bird_type = sess.BirdType.STARLING        
    elif session_dict['bird_type'] == 'ZEBRA':     sess.bird_type = sess.BirdType.ZEBRA        
    elif session_dict['bird_type'] == 'BENGALESE': sess.bird_type = sess.BirdType.BENGALESE   
    else:                                          sess.bird_type = sess.BirdType.UNKNOWN_BIRDTYPE       # Also if user input is unrecognized  
if 'bird_sex' in session_dict: # UNKNOWN_BIRDSEX(0), MALE(1), FEMALE(2)          
    if session_dict['bird_sex'] == 'MALE':     sess.bird_sex = sess.BirdSex.MALE                     
    elif session_dict['bird_sex'] == 'FEMALE': sess.bird_sex = sess.BirdSex.FEMALE   
    else:                                      sess.bird_sex = sess.BirdSex.UNKNOWN_BIRDSEX              # Also if user input is unrecognized      
if 'condition' in session_dict: # UNKNOWN_CONDITION(0), HABITUATION(1), CHRONIC(2), ACUTE(3)        
    if   session_dict['condition'] == 'HABITUATION': sess.condition = sess.Condition.HABITUATION        
    elif session_dict['condition'] == 'CHRONIC':     sess.condition = sess.Condition.CHRONIC   
    elif session_dict['condition'] == 'ACUTE':       sess.condition = sess.Condition.ACUTE 
    else:                                            sess.condition = sess.Condition.UNKNOWN_CONDITION   # Also if user input is unrecognized      
if 'bird_uid'           in session_dict: sess.bird_uid            = session_dict['bird_uid']                # string e.g. z_m10g8_20
if 'weight_grams'       in session_dict: sess.weight_grams        = session_dict['weight_grams']            # float
if 'testosterone'       in session_dict: sess.testosterone        = session_dict['testosterone']            # bool: True / False
if 'testosterone_date'  in session_dict: sess.testosterone_date   = session_dict['testosterone_date']       # string: e.g. 2021-03-10
if 'dummy_weight'       in session_dict: sess.dummy_weight        = session_dict['dummy_weight']            # bool: True / False
if 'dummy_weight_grams' in session_dict: sess.dummy_weight_grams  = session_dict['dummy_weight_grams']      # bool: True / False
if 'dummy_weight_date'  in session_dict: sess.dummy_weight_date   = session_dict['dummy_weight_date']       # string: e.g. 2021-03-10
if 'dummy_tether'       in session_dict: sess.dummy_tether        = session_dict['dummy_tether']            # bool: True / False
if 'dummy_tether_date'  in session_dict: sess.dummy_tether_date   = session_dict['dummy_tether_date']       # string: e.g. 2021-03-10
if 'box'                in session_dict: sess.box                 = session_dict['box']                     # string: e.g. passaro1, cuervecito3, shoox
if 'details'            in session_dict: 
    for det in session_dict['details']: sess.details.append(det)                                         # repeated string: (Any additional info)
sess.date = str(datetime.now(pytz.timezone('US/Pacific')).date())                                        # string: e.g. 2021-03-10
sess.time = str(datetime.now(pytz.timezone('US/Pacific')).time())                                        # string: e.g. 14:42:01.754603 (microseconds precision)
sess.sess_uid = sess.Condition.keys()[sess.condition] + '-' + sess.bird_uid + '-' + datetime.now(pytz.timezone('US/Pacific')).strftime("%Y%m%d-%H:%M:%S")    # string: e.g. habituation_birdID_date_time


# Iterate over ACQUISITIONS 
if 'acquisitions' in session_dict:
    for acquisition_dict in session_dict['acquisitions']:

        acquisition = sess.acquisitions.add()
        if 'acquisition_hardware' in acquisition_dict: acquisition.acquisition_hardware = acquisition_dict['acquisition_hardware']  # openephys, intan, spikeglx, uma8, raspi, any custom amp
        if 'acquisition_software' in acquisition_dict: acquisition.acquisition_software = acquisition_dict['acquisition_software']  # openephys, openephys++, spikeglx, alsa, vlc

        # Iterate over Sensors
        if 'sensors' in acquisition_dict:
            for sensor_dict in acquisition_dict['sensors']:
                sensor = acquisition.sensors.add()
                if 'acquisition_signal' in sensor_dict: sensor.acquisition_signal = sensor_dict['acquisition_signal'] # audio, audio, pressure, sync, emg, video                  
                if 'manufacturer'       in sensor_dict: sensor.manufacturer       = sensor_dict['manufacturer']       # miniDSP, earthworks, fujikura, inhouse, inhouse
                if 'model'              in sensor_dict: sensor.model              = sensor_dict['model']              # uma8, m30, fhm-20, uma8_sync, emg_amp_inhouse
                if 'serial_number'      in sensor_dict: sensor.serial_number      = sensor_dict['serial_number']      # '', '', '', uma_syn_001, emg_amp_001
                if 'signal_name'        in sensor_dict: sensor.signal_name        = sensor_dict['signal_name']        # uma_syn_001, pressure, syn_uma8, emg, syn_stim, stim, …
                if 'headstage'          in sensor_dict: sensor.headstage          = sensor_dict['headstage']          # '', '', intan32, ", intan32
                if 'channel_group'      in sensor_dict: sensor.channel_group      = sensor_dict['channel_group']      # '', AIN, port_0, DIN, port_1
                if 'channels'           in sensor_dict: sensor.channels           = sensor_dict['channels']           # '', [0], [aux_0], [0], [aux_0, aux_1]
                if 'locations'          in sensor_dict: sensor.locations          = sensor_dict['locations']          # left, right, tracheal rings, muscles...
                if 'details'            in sensor_dict: 
                    for det in sensor_dict['details']: sensor.details.append(det) # repeated string

        #Iterate over Neural probes
        if 'neuralprobes' in acquisition_dict:
            for neural_probe_dict in acquisition_dict['neuralprobes']:
                neural_probe = acquisition.neuralprobes.add()
                if 'acquisition_signal'          in neural_probe_dict: neural_probe.acquisition_signal          = neural_probe_dict['acquisition_signal']          # string: e.g. neural
                if 'manufacturer'                in neural_probe_dict: neural_probe.manufacturer                = neural_probe_dict['manufacturer']                # string: neuronexus, neuropixel, masmanidis, inhouse
                if 'model'                       in neural_probe_dict: neural_probe.model                       = neural_probe_dict['model']                       # string: buzsaki32, neuropixels_1
                if 'serial_number'               in neural_probe_dict: neural_probe.serial_number               = neural_probe_dict['serial_number']               # string: U656
                if 'num_channels'                in neural_probe_dict: neural_probe.num_channels                = neural_probe_dict['num_channels']                # int32: 16, 32, 64, 384
                if 'tip_depth_microns'           in neural_probe_dict: neural_probe.tip_depth_microns           = neural_probe_dict['tip_depth_microns']           # float: depth of the tip of the probe
                if 'implant_coordinates_microns' in neural_probe_dict: neural_probe.implant_coordinates_microns = neural_probe_dict['implant_coordinates_microns'] # string: A/P, M/L, D/V
                if 'hemisphere'                  in neural_probe_dict: neural_probe.hemisphere                  = neural_probe_dict['hemisphere']                  # string: left, right
                if 'brain_nucleus'               in neural_probe_dict: 
                    for bn in neural_probe_dict['brain_nucleus']:      neural_probe.brain_nucleus.append(bn)                                                       # repeated string [hvc], [hvc,ra], area_X
                if 'headstage'                   in neural_probe_dict: neural_probe.headstage                   = neural_probe_dict['headstage']                   # string: e.g. H32
                if 'channel_group'               in neural_probe_dict: neural_probe.channel_group               = neural_probe_dict['channel_group']               # string: port_0, imec_0
                if 'channels'                    in neural_probe_dict: neural_probe.channels                    = neural_probe_dict['channels']                    # string: e.g. 1-64,
                if 'details'                     in neural_probe_dict: 
                    for det in neural_probe_dict['details']: neural_probe.details.append(det)                                                                      # repeated string

        # Iterate over Stimuli
        if 'stimuli' in acquisition_dict:
            for stimulus_dict in acquisition_dict['stimuli']:
                stimulus = acquisition.stimuli.add()
                if 'stimulus_signal' in stimulus_dict: stimulus.stimulus_signal = stimulus_dict['stimulus_signal'] # string: female, video, song_replay, neural_stim
                if 'manufacturer'    in stimulus_dict: stimulus.manufacturer    = stimulus_dict['manufacturer']    # string: '', inhouse, inhouse, neuronexus
                if 'model'           in stimulus_dict: stimulus.model           = stimulus_dict['model']           # string: '', '', '3 females in cage, buzsaki32
                if 'serial_number'   in stimulus_dict: stimulus.serial_number   = stimulus_dict['serial_number']   #
                if 'signal_name'     in stimulus_dict: stimulus.signal_name     = stimulus_dict['signal_name']
                if 'channel_gropup'  in stimulus_dict: stimulus.channel_gropup  = stimulus_dict['channel_gropup']  # string: AIN, port_0, DIN, port_1
                if 'channels'        in stimulus_dict: stimulus.channels        = stimulus_dict['channels']        # string: [0], [aux_0], [0], [aux_0, aux_1]
                if 'details'         in stimulus_dict: 
                    for det in stimulus_dict['details']: stimulus.details.append(det)                              # repeated string


In [3]:
sess

bird_type: ZEBRA
bird_sex: MALE
bird_uid: "z_m10g8_20"
date: "2021-04-28"
time: "11:24:33.710174"
weight_grams: 18.5
dummy_weight_grams: 0.6000000238418579
dummy_weight_date: "2021-03-10"
condition: HABITUATION
sess_uid: "HABITUATION-z_m10g8_20-20210428-11:24:33"
box: "cuervecito3"
details: "dummy_weight + tether"
acquisitions {
  acquisition_hardware: "uma8-usb"
  acquisition_software: "alsa"
  sensors {
    acquisition_signal: "audio"
    manufacturer: "miniDSP"
    model: "uma8raw"
    signal_name: "audio_raw"
    channels: "0-6"
    locations: "OUT"
    details: "Positioned in top-back-left and top-front-right corners of the chamber."
    details: "detalis2"
  }
  sensors {
    acquisition_signal: "audio"
    manufacturer: "miniDSP"
    model: "uma8DSP"
    signal_name: "audio_DSP"
    channels: "7-8"
    locations: "OUT"
    details: "Positioned in top-back-left and top-front-right corners of the chamber."
    details: "detalis2"
  }
}

In [4]:
# Write serialized, binary file with message.

import sys
fp = open('metadata.pb', "wb")
fp.write(sess.SerializeToString())
fp.close()


In [5]:
# Write human-readable JSON file with message.

import json
from google.protobuf.json_format import MessageToDict

json_obj = MessageToDict(sess, including_default_value_fields=False, preserving_proto_field_name=False, use_integers_for_enums=False, descriptor_pool=None, float_precision=None)

with open('metadata.json', 'w') as fj:
    json.dump(json_obj, fj, indent=5)
fj.close()

In [6]:
json_obj

{'birdType': 'ZEBRA',
 'birdSex': 'MALE',
 'birdUid': 'z_m10g8_20',
 'date': '2021-04-28',
 'time': '11:24:33.710174',
 'weightGrams': 18.5,
 'dummyWeightGrams': 0.6,
 'dummyWeightDate': '2021-03-10',
 'condition': 'HABITUATION',
 'sessUid': 'HABITUATION-z_m10g8_20-20210428-11:24:33',
 'box': 'cuervecito3',
 'details': ['dummy_weight + tether'],
 'acquisitions': [{'acquisitionHardware': 'uma8-usb',
   'acquisitionSoftware': 'alsa',
   'sensors': [{'acquisitionSignal': 'audio',
     'manufacturer': 'miniDSP',
     'model': 'uma8raw',
     'signalName': 'audio_raw',
     'channels': '0-6',
     'locations': 'OUT',
     'details': ['Positioned in top-back-left and top-front-right corners of the chamber.',
      'detalis2']},
    {'acquisitionSignal': 'audio',
     'manufacturer': 'miniDSP',
     'model': 'uma8DSP',
     'signalName': 'audio_DSP',
     'channels': '7-8',
     'locations': 'OUT',
     'details': ['Positioned in top-back-left and top-front-right corners of the chamber.',
   